In [5]:
import cv2
import numpy as np
from util import get_parking_spots_bboxes,empty_or_not

In [6]:
def calc_diff(im1, im2):
    """
    두 이미지 간의 차이를 계산합니다.
    
    인수:
        im1: 첫 번째 이미지
        im2: 두 번째 이미지

    반환값:
        두 이미지의 평균 차이
    """
    return np.abs(np.mean(im1) - np.mean(im2))

In [7]:
mask = 'mask_1920_1080.png'
video_path = 'parking_1920_1080_loop.mp4'

In [8]:
mask = cv2.imread(mask,0) #마스크 이미지를 그레이스케일로 읽는다

In [9]:
cap = cv2.VideoCapture(video_path) #비디오 파일을 열고 읽는다

In [10]:
connected_components = cv2.connectedComponentsWithStats(mask, 4, cv2.CV_32S) #마스크 이미지에서 연결된 컴포넌트를 분석한다

In [11]:
spots = get_parking_spots_bboxes(connected_components) #주차 공간 상태와 차이 리스트를 초기화한다

In [12]:
# 주차 공간 상태와 차이 리스트를 초기화합니다
spots_status = [None for j in spots]
diffs = [None for j in spots]

previous_frame = None

frame_nmr = 0
ret = True
step = 30

In [ ]:
while ret:
    #비디오에서 프레임을 읽준다
    ret, frame = cap.read()

    #설정된 단계마다 프레임을 처리한다
    if frame_nmr % step == 0 and previous_frame is not None:
        for spot_indx, spot in enumerate(spots):
            x1, y1, w, h = spot

            #현재 프레임에서 주차 공간을 잘라낸다
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]

            #이전 프레임과의 차이를 계산한다
            diffs[spot_indx] = calc_diff(spot_crop, previous_frame[y1:y1 + h, x1:x1 + w, :])

        #차이 리스트를 출력합니다.
        print([diffs[j] for j in np.argsort(diffs)][::-1])

    #설정된 단계마다 주차 공간의 상태를 업데이트한다
    if frame_nmr % step == 0:
        if previous_frame is None:
            arr_ = range(len(spots))
        else:
            arr_ = [j for j in np.argsort(diffs) if diffs[j] / np.amax(diffs) > 0.4]
        for spot_indx in arr_:
            spot = spots[spot_indx]
            x1, y1, w, h = spot

            #현재 프레임에서 주차 공간을 잘라낸다
            spot_crop = frame[y1:y1 + h, x1:x1 + w, :]
            spot_status = empty_or_not(spot_crop)
            spots_status[spot_indx] = spot_status

    #설정된 단계마다 이전 프레임을 업데이트
    if frame_nmr % step == 0:
        previous_frame = frame.copy()

    #프레임에 주차 공간의 경계를 표시
    for spot_indx, spot in enumerate(spots):
        spot_status = spots_status[spot_indx]
        x1, y1, w, h = spots[spot_indx]

        if spot_status:
            #주차 공간이 비어있으면 녹색으로 표시
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 255, 0), 2)
        else:
            # 주차 공간이 가득 차면 빨간색으로 표시
            frame = cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)

    #화면에 주차 공간의 개수를 표시
    cv2.rectangle(frame, (80, 20), (550, 80), (0, 0, 0), -1)
    cv2.putText(frame, 'Available spots: {} / {}'.format(str(sum(spots_status)), str(len(spots_status))), (100, 60),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    #프레임을 표시
    cv2.namedWindow('frame', cv2.WINDOW_NORMAL)
    cv2.imshow('frame', frame)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

    frame_nmr += 1


cap.release()
cv2.destroyAllWindows()